In [1]:
from src.gather_vote_texts import gather_vote_texts, download_vote_texts
import pandas as pd


# votes = pd.read_parquet("data/parquet/votes.parquet")
# download_vote_texts(votes.iloc[0])

gather_vote_texts()

2025-05-18 13:47:23.047 | INFO     | src.gather_vote_texts:gather_vote_texts:137 - Gathering vote texts from Bundestag website...
2025-05-18 13:47:27.086 | ERROR    | src.gather_vote_texts:gather_vote_texts:149 - Vote data: id                                                20230615_3
name                         Wahl zum Europäischen Parlament
date                                     2023-06-15 00:00:00
xls_url    https://www.bundestag.de/resource/blob/953456/...
pdf_url    https://www.bundestag.de/resource/blob/953458/...
Name: 89, dtype: object
2025-05-18 13:47:27.331 | ERROR    | src.gather_vote_texts:download_vote_texts:109 - Failed to extract text from data/tmp/vote_pdf/20221215_1/drucksachen/0.pdf
2025-05-18 13:47:27.705 | ERROR    | src.gather_vote_texts:download_vote_texts:117 - Failed to extract drucksache or date from data/tmp/vote_pdf/20221123_1/drucksachen/1.pdf drucksache: None date: 05.08.2022
2025-05-18 13:47:33.698 | ERROR    | src.gather_vote_texts:gather_vote_texts:14

In [2]:
texts = pd.read_parquet("data/parquet/vote_texts.parquet")

In [6]:
def count_gesetzes_entwürfe(documents: pd.DataFrame) -> int:
    return (documents["document_type"] == "Gesetzentwurf").sum()

texts.groupby("vote_id").apply(count_gesetzes_entwürfe).idxmax()

'20130613'

In [7]:
texts.query("vote_id == '20130613'")

,vote_id,filename,drucksache,date,document_type
1656,20130613,data/tmp/vote_pdf/20130613/drucksachen/0.pdf,17_13829,10.06.2013,Gesetzentwurf
1657,20130613,data/tmp/vote_pdf/20130613/drucksachen/1.pdf,17_13901,12.06.2013,12. 06. 2013
1658,20130613,data/tmp/vote_pdf/20130613/drucksachen/2.pdf,17_13470,14.05.2013,Gesetzentwurf
1659,20130613,data/tmp/vote_pdf/20130613/drucksachen/3.pdf,17_13961,12.06.2013,Beschlussempfehlung und Bericht
1660,20130613,data/tmp/vote_pdf/20130613/drucksachen/4.pdf,17_13965,12.06.2013,Entschließungsantrag


In [10]:
texts["drucksache"].value_counts()

20_13413    9
18_11131    8
18_12588    8
20_12805    6
19_28692    6
           ..
19_23310    1
19_23320    1
19_23313    1
19_23312    1
17_11053    1
Name: drucksache, Length: 1394, dtype: int64